In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.max_columns", None)
sns.set_style("whitegrid")

CURRENT_DIR = Path.cwd().resolve()
if (CURRENT_DIR / "data").exists():
    ROOT_DIR = CURRENT_DIR
else:
    ROOT_DIR = CURRENT_DIR.parent

if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

In [ ]:
RAW_DATA_DIR = ROOT_DIR / "data" / "raw"

train = pd.read_csv(RAW_DATA_DIR / "TRAIN.csv")
test = pd.read_csv(RAW_DATA_DIR / "TEST.csv")

train["id"] = train["id"].astype(str)
test["id"] = test["id"].astype(str)

In [3]:
train["date"] = pd.to_datetime(train["date"])
test["date"]  = pd.to_datetime(test["date"])

train["sale_year"]  = train["date"].dt.year
train["sale_month"] = train["date"].dt.month

test["sale_year"]  = test["date"].dt.year
test["sale_month"] = test["date"].dt.month

train = train.drop(columns=["date"])
test  = test.drop(columns=["date"])


In [4]:
train["price_log"] = np.log1p(train["price"])


In [5]:
train["basement_ratio"] = np.where(
    train["sqft_living"] == 0,
    0,
    train["sqft_basement"] / train["sqft_living"]
 )
test["basement_ratio"] = np.where(
    test["sqft_living"] == 0,
    0,
    test["sqft_basement"] / test["sqft_living"]
 )


In [6]:
train["living_lot_ratio"] = np.where(
    train["sqft_lot"] == 0,
    0,
    train["sqft_living"] / train["sqft_lot"]
 )
test["living_lot_ratio"] = np.where(
    test["sqft_lot"] == 0,
    0,
    test["sqft_living"] / test["sqft_lot"]
 )


In [7]:
train["living_vs_neighbors"] = np.where(
    train["sqft_living15"] == 0,
    0,
    train["sqft_living"] / train["sqft_living15"]
 )
test["living_vs_neighbors"] = np.where(
    test["sqft_living15"] == 0,
    0,
    test["sqft_living"] / test["sqft_living15"]
 )


In [8]:
CURRENT_YEAR = int(train["sale_year"].max())

train["house_age"] = (CURRENT_YEAR - train["yr_built"]).clip(lower=0)
test["house_age"] = (CURRENT_YEAR - test["yr_built"]).clip(lower=0)

train["is_renovated"] = (train["yr_renovated"] > 0).astype(int)
test["is_renovated"] = (test["yr_renovated"] > 0).astype(int)


In [9]:
drop_cols = ["yr_built", "yr_renovated"]

train = train.drop(columns=drop_cols)
test  = test.drop(columns=drop_cols)


In [10]:
TARGET = "price_log"

X = train.drop(columns=["price", "price_log"])
y = train[TARGET]


In [11]:
processed_train_path = ROOT_DIR / "data" / "train_processed.csv"
processed_test_path = ROOT_DIR / "data" / "test_processed.csv"

train.to_csv(processed_train_path, index=False)
test.to_csv(processed_test_path, index=False)

processed_train_path, processed_test_path

(WindowsPath('D:/ML_PROJECTS/real-estate-multimodal/data/train_processed.csv'),
 WindowsPath('D:/ML_PROJECTS/real-estate-multimodal/data/test_processed.csv'))

In [12]:
X

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,zipcode,lat,long,sqft_living15,sqft_lot15,sale_year,sale_month,basement_ratio,living_lot_ratio,living_vs_neighbors,house_age,is_renovated
0,9117000170,4,2.25,1810,9240,2.0,0,0,3,7,1810,0,98055,47.4362,-122.187,1660,9240,2015,5,0.000000,0.195887,1.090361,54,0
1,6700390210,3,2.50,1600,2788,2.0,0,0,4,7,1600,0,98031,47.4034,-122.187,1720,3605,2014,7,0.000000,0.573888,0.930233,23,0
2,7212660540,4,2.50,1720,8638,2.0,0,0,3,8,1720,0,98003,47.2704,-122.313,1870,7455,2015,1,0.000000,0.199120,0.919786,21,0
3,8562780200,2,2.25,1240,705,2.0,0,0,3,7,1150,90,98027,47.5321,-122.073,1240,750,2015,4,0.072581,1.758865,1.000000,6,0
4,7760400350,3,2.00,1280,13356,1.0,0,0,3,7,1280,0,98042,47.3715,-122.074,1590,8071,2014,12,0.000000,0.095837,0.805031,21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16204,5272200045,3,1.50,1000,6914,1.0,0,0,3,7,1000,0,98125,47.7144,-122.319,1000,6947,2014,11,0.000000,0.144634,1.000000,68,0
16205,9578500790,3,2.50,3087,5002,2.0,0,0,3,8,3087,0,98023,47.2974,-122.349,2927,5183,2014,11,0.000000,0.617153,1.054663,1,0
16206,7202350480,3,2.50,2120,4780,2.0,0,0,3,7,2120,0,98053,47.6810,-122.032,1690,2650,2014,9,0.000000,0.443515,1.254438,11,0
16207,1723049033,1,0.75,380,15000,1.0,0,0,3,5,380,0,98168,47.4810,-122.323,1170,15000,2014,6,0.000000,0.025333,0.324786,52,0


In [13]:
y

0        12.501142
1        12.409018
2        12.206078
3        12.772806
4        12.354497
           ...    
16204    12.842652
16205    12.899097
16206    13.262127
16207    12.409018
16208    12.660331
Name: price_log, Length: 16209, dtype: float64